## Word Embedding dataset

En este notebook se va a convertir los tokens de los textos en un vector promedio de los embeddings de las palabras. Para ello se va a utilizar el modelo de embeddings de palabras preentrenadas en el notebook del paso anterior.

Al final tendremos el dataset con 201 columnas -> 200 columnas de embeddings y 1 columna de la variable objetivo.

In [ ]:
# Instalación de librerías
%pip install -r '../requirements.txt'

In [ ]:
# Importación de librerías
import pandas as pd
import json
from gensim.models import Word2Vec
import numpy as np

In [ ]:
# Importación de datos
datos_entrenamiento = pd.read_csv('../3-NLP/dist/dataset-spanish-tweets-train-preprocessed.csv', delimiter=',', encoding='utf-8')
datos_prueba = pd.read_csv('../3-NLP/dist/dataset-spanish-tweets-test-preprocessed.csv', delimiter=',', encoding='utf-8')
datos_validacion = pd.read_csv('../3-NLP/dist/dataset-spanish-tweets-validation-preprocessed.csv', delimiter=',', encoding='utf-8')
datos_entrenamiento['text'] = datos_entrenamiento['text'].apply(lambda x: json.loads(x))
datos_prueba['text'] = datos_prueba['text'].apply(lambda x: json.loads(x))
datos_validacion['text'] = datos_validacion['text'].apply(lambda x: json.loads(x))

In [ ]:
# Importación modelo
modelo = Word2Vec.load('../4-word-embedding/dist/word2vec.model')
unk_vector = modelo.wv.vectors.mean(axis=0)

In [ ]:
def convert_text_to_vector_number(texto):
    arreglo_vectores_palabra = []
    for palabra in texto:
        if palabra in modelo.wv:
            arreglo_vectores_palabra.append(modelo.wv[palabra])
        else:
            arreglo_vectores_palabra.append(unk_vector)
    # Calcular el promedio de los vectores
    arreglo_vectores_palabra = np.mean(arreglo_vectores_palabra, axis=0)
    return arreglo_vectores_palabra

### Dataset entrenamiento

In [ ]:
# Aplicar la función de conversión a cada texto en el dataset
datos_entrenamiento['text'] = datos_entrenamiento['text'].apply(
    convert_text_to_vector_number)

# Expandir la lista de vectores en columnas separadas
expanded_columns = pd.DataFrame(datos_entrenamiento['text'].tolist())

# Renombrar las columnas a dim_1, dim_2, ..., dim_200
expanded_columns.columns = [
    f'dim_{i+1}' for i in range(expanded_columns.shape[1])]

# Concatenar las columnas expandidas con la columna 'sentiment'
final_dataset_entrenamiento = pd.concat(
    [expanded_columns, datos_entrenamiento['sentiment']], axis=1)

# Mapear los valores de 'sentiment' a números
# 0 -> negative, 1 -> positive
final_dataset_entrenamiento['sentiment'] = final_dataset_entrenamiento['sentiment'].map(
    {'negative': 0, 'positive': 1})

# Guardar el dataset final en un archivo CSV
final_dataset_entrenamiento.to_csv('./dist/dataset-train.csv', index=False)

### Dataset prueba

In [ ]:
datos_prueba['text'] = datos_prueba['text'].apply(
    convert_text_to_vector_number)
expanded_columns = pd.DataFrame(datos_prueba['text'].tolist())

# Renombrar las columnas a dim_1, dim_2, ..., dim_200
expanded_columns.columns = [
    f'dim_{i+1}' for i in range(expanded_columns.shape[1])]

# Concatenar las columnas expandidas con la columna 'sentiment'
final_dataset_prueba = pd.concat(
    [expanded_columns, datos_prueba['sentiment']], axis=1)
# 0 -> negative, 1 -> positive
final_dataset_prueba['sentiment'] = final_dataset_prueba['sentiment'].map(
    {'negative': 0, 'positive': 1, })
final_dataset_prueba.to_csv('./dist/dataset-test.csv', index=False)

### Dataset validación

In [ ]:
datos_validacion['text'] = datos_validacion['text'].apply(
    convert_text_to_vector_number)
expanded_columns = pd.DataFrame(datos_validacion['text'].tolist())

# Renombrar las columnas a dim_1, dim_2, ..., dim_200
expanded_columns.columns = [
    f'dim_{i+1}' for i in range(expanded_columns.shape[1])]

# Concatenar las columnas expandidas con la columna 'sentiment'
final_dataset_validacion = pd.concat(
    [expanded_columns, datos_validacion['sentiment']], axis=1)
# 0 -> negative, 1 -> positive
final_dataset_validacion['sentiment'] = final_dataset_validacion['sentiment'].map(
    {'negative': 0, 'positive': 1})
final_dataset_validacion.to_csv('./dist/dataset-validation.csv', index=False)